# Starter Notebook (to make the following pulls):
## WHO Mortality and Population data
## World Bank GDP data
## ICD-10 Codes

In [2]:
import pandas as pd
import numpy
import csv

In [3]:
# WHO Mortality data
# Part 1 - https://www.who.int/healthinfo/statistics/Morticd10_part1.zip?ua=1
# Part 2 - https://www.who.int/healthinfo/statistics/Morticd10_part2.zip?ua=1

mortality_part1 = pd.read_csv('Morticd10_part1.csv', low_memory = False)
mortality_part2 = pd.read_csv('Morticd10_part2.csv', low_memory = False)
mortality = pd.concat([mortality_part1, mortality_part2], sort = False)

# WHO Country Codes
# https://www.who.int/healthinfo/statistics/country_codes.zip?ua=1

country_codes = pd.read_csv('country_codes.csv', low_memory = False)

# WHO Population data
# https://www.who.int/healthinfo/Pop.zip?ua=1

# population = pd.read_csv('pop.csv', low_memory = False)


In [4]:
#Filtered by the year 2016 only
mortality_2012 = mortality[mortality["Year"]==2012]
# name_dump = []

# for code in mortality_2016["Country"]:
#     name = country_codes[country_codes['country'] == code]
#     name_dump.append(name.iloc[0][1])
    
# mortality_2016['Country Name'] = pd.Series(name_dump)
# mortality_2012.loc[mortality_2012['Cause'] == 'P22']

In [5]:
# #  table = pivot_table(df, values='D', index=['A', 'B'],columns=['C'], aggfunc=np.sum)
# import numpy as np
# table = pd.pivot_table(mortality, values = "Deaths2",
#                    columns = "Country",index=["Cause"],
#                    aggfunc=np.sum)

In [17]:
#Groupby Country and Cause and select top 3 causes of death for children <1 for each country
Deaths2_group_2012 = pd.DataFrame(mortality_2012.groupby(['Country','Cause'])['Deaths2'].sum())
Deaths2_non_null_2012 = Deaths2_group_2012[Deaths2_group_2012["Deaths2"]!=0].copy()
Deaths2_non_null_2012.sort_values(["Country","Deaths2"], ascending=False,inplace=True)
Deaths2_non_null_2012=Deaths2_non_null_2012.reset_index()
Result_2012 = Deaths2_non_null_2012.groupby(["Country"]).head(4)
Result_2012 = Result_2012.reset_index()

Result_2012["Infant Death %"] = 0

for i, row in Result_2012.iterrows():
   if row['Cause'] == "AAA":
       overall_deaths = row["Deaths2"]
       Result_2012.iloc[i, 4] = 1
   else:
       Result_2012.iloc[i, 4] = row["Deaths2"]/overall_deaths

Result_2012.head(9)

,index,Country,Cause,Deaths2,Infant Death %
0,0,5150,AAA,294.0,1.000000
1,1,5150,P072,61.0,0.207483
2,2,5150,R95,18.0,0.061224
3,3,5150,W759,17.0,0.057823
4,118,5070,AAA,320.0,1.000000
5,119,5070,P073,39.0,0.121875
6,120,5070,A099,22.0,0.068750
7,121,5070,P072,16.0,0.050000
8,233,5020,AAA,1031.0,1.000000


In [11]:
# Getting the Country names in it

mydict = {}
with open('country_codes.csv', mode='r') as infile:
    reader = csv.reader(infile, delimiter = ',')
    for row in reader:
        mydict[row[0]] = row[1]

Result_2012['Country_Name'] = Result_2012['Country'].astype(str).replace(mydict)

Result_2012.head()

,index,Country,Cause,Deaths2,Infant Death %,Country_Name
0,0,5150,AAA,294.0,1.000000,New Zealand
1,1,5150,P072,61.0,0.207483,New Zealand
2,2,5150,R95,18.0,0.061224,New Zealand
3,3,5150,W759,17.0,0.057823,New Zealand
4,118,5070,AAA,320.0,1.000000,Fiji


In [12]:
#Getting it down to one row per (Total infant deaths)

Result_2012_b = Result_2012[Result_2012["Cause"]=='AAA']
Result_2012_c = Result_2012_b[['index', 'Country', 'Cause', 'Deaths2', 'Infant Death %', 'Country_Name']]
Result_2012_c.head()

,index,Country,Cause,Deaths2,Infant Death %,Country_Name
0,0,5150,AAA,294.0,1.0,New Zealand
4,118,5070,AAA,320.0,1.0,Fiji
8,233,5020,AAA,1031.0,1.0,Australia
12,479,4335,AAA,6390.0,1.0,Uzbekistan
16,637,4330,AAA,217.0,1.0,"United Kingdom, Scotland"


In [ ]:
Result_2012_b = Result_2012[Result_2012["Cause"]=='AAA']
Result_2012_c = Result_2012_b[['index', 'Country', 'Cause', 'Deaths2', 'Infant Death %', 'Country_Name']]
Result_2012_c.head()

In [42]:
#Getting it to have Both Deaths2 and Deaths1
#    WE MUST ADD AN ADDITIONAL COLUMN: Deaths2 divided by Deaths1

Deaths4 = Result_2012_c.merge(Deaths3_group, on=["Country"])
Deaths4.head()

,index,Country,Cause,Deaths2,Infant Death %,Country_Name,Deaths1
0,0,5150,AAA,294.0,1.0,New Zealand,60554
1,118,5070,AAA,320.0,1.0,Fiji,13532
2,233,5020,AAA,1031.0,1.0,Australia,294196
3,479,4335,AAA,6390.0,1.0,Uzbekistan,291976
4,637,4330,AAA,217.0,1.0,"United Kingdom, Scotland",109874


In [16]:
Result3 = Result_2012.groupby(["Cause"]).sum()['Deaths2']

Result4 = pd.DataFrame(Result3).sort_values('Deaths2', ascending = False)

Result4.head()

,Deaths2
Cause,
AAA,293923.0
1000,28692.0
1092,15554.0
P220,14976.0
P369,10695.0


In [ ]:
mydict = {}
with open('country_codes.csv', mode='r') as infile:
    reader = csv.reader(infile, delimiter = ',')
    for row in reader:
        mydict[row[0]] = row[1]
        
Result3['Country_Name'] = Result['Country'].astype(str).replace(mydict)
Result4 = pd.DataFrame(Result3)
Result4 = Result3.sort_values('Deaths2', ascending = False).reset_index()

In [18]:
#This gets us The country names and some of the cause names

mydict = {}
mydict2 = {"AAA":"Total"}

with open('country_codes.csv', mode='r') as infile:
    reader = csv.reader(infile, delimiter = ',')
    for row in reader:
        mydict[row[0]] = row[1]

with open('icd10_codes_2011_csv.csv', mode='r') as infile:
    reader2 = csv.reader(infile, delimiter = ',')
    for row in reader2:
        mydict2[row[0]] = row[1]
        
Result['Cause_Name'] = Result['Cause'].replace(mydict2)
Result['Country_Name'] = Result['Country'].astype(str).replace(mydict)

Result.head(7)

NameError: name 'Result' is not defined

In [23]:
gdp = pd.read_csv('GDP.csv')

gdp_2012 = gdp[['Country Name', '2012']].copy()

gdp_2012.rename({'2012' : 'GDP (per capita, in current USD)'},

                axis = 1,

                inplace = True)

gdp_2012['Quantile'] = pd.qcut(gdp_2012['GDP (per capita, in current USD)'], 5, ['Q1', 'Q2', 'Q3', 'Q4', 'Q5'])


new_df = gdp_2012

excluded_words = ['dividend',
                  'world',
                  'small',
                  'IDA',
                  'IBRD',
                  'indebted',
                  'poor',
                  'conflict',
                  'developed',
                  'OECD',
                  'income']

for word in excluded_words:

    new_df = new_df[~new_df["Country Name"].str.contains(word)]

new_df.dropna(how='all')


gdp_2012.head(30)

,Country Name,"GDP (per capita, in current USD)",Quantile
0,Aruba,24709.602260,Q4
1,Afghanistan,648.511070,Q1
2,Angola,5102.489969,Q3
3,Albania,4247.614342,Q2
4,Andorra,38391.080870,Q5
5,Arab World,7465.135230,Q3
6,United Arab Emirates,42086.689950,Q5
7,Argentina,12969.707120,Q4
8,Armenia,3684.804810,Q2
9,American Samoa,11660.329530,Q4


In [13]:
mydict = {}
with open('country_codes.csv', mode='r') as infile:
    reader = csv.reader(infile, delimiter = ',')
    for row in reader:
        mydict[row[0]] = row[1]

Result_2012['Country_Name'] = Result_2012['Country'].astype(str).replace(mydict)

Result_2012.head(20)

,index,Country,Cause,Deaths2,Infant Death %,Country_Name
0,0,5150,AAA,294.0,1.000000,New Zealand
1,1,5150,P072,61.0,0.207483,New Zealand
2,2,5150,R95,18.0,0.061224,New Zealand
3,3,5150,W759,17.0,0.057823,New Zealand
4,118,5070,AAA,320.0,1.000000,Fiji
5,119,5070,P073,39.0,0.121875,Fiji
6,120,5070,A099,22.0,0.068750,Fiji
7,121,5070,P072,16.0,0.050000,Fiji
8,233,5020,AAA,1031.0,1.000000,Australia
9,234,5020,R95,58.0,0.056256,Australia


In [16]:
Result_2012_b = Result_2012[Result_2012["Cause"]=='AAA']
Result_2012_c = Result_2012_b[['index', 'Deaths2', 'Country_Name']]
Result_2012_c.head()

,index,Deaths2,Country_Name
0,0,294.0,New Zealand
4,118,320.0,Fiji
8,233,1031.0,Australia
12,479,6390.0,Uzbekistan
16,637,217.0,"United Kingdom, Scotland"


In [21]:
Deaths2_group = pd.DataFrame(mortality_2016.groupby(['Country','Cause'])[['Deaths2','Deaths1']].sum())
Deaths2_non_null = Deaths2_group[Deaths2_group["Deaths2"]!=0]
Deaths2_group.sort_values(["Country","Deaths2"], ascending=False,inplace=True)
Deaths2_group=Deaths2_group.reset_index()
Result = Deaths2_group.groupby(["Country"]).head(4)
Result.head()

,Country,Cause,Deaths2,Deaths1
0,4335,AAA,7764.0,154791
1,4335,P22,1020.0,1020
2,4335,P23,927.0,927
3,4335,J18,803.0,2244
816,4330,AAA,181.0,56728


In [25]:
# A peek at the GDP dataset



In [ ]:
Deaths3_group = pd.DataFrame(mortality_2012.groupby(['Country'])['Deaths1'].sum())
Deaths3_group.head(10)

Causes_count_dict = {}

for ix, row in Result.iterrows():
    if row['Cause'] not in Causes_count_dict:
        Causes_count_dict[row['Cause']] = row['Deaths2']
    if row['Cause'] in Causes_count_dict:
        Causes_count_dict[row['Cause']] += row['Deaths2']
        
Causes_count2 = {"AAA": [12634]}
Result2 = Result.groupby(["Cause","Country"]).sum()
Result2

pd.DataFrame(Causes_count2)a

#Groupby Country and Cause and select top 3 causes of death for children <1 for each country
Deaths2_group_2012 = pd.DataFrame(mortality_2012.groupby(['Country','Cause'])['Deaths2'].sum())
Deaths2_non_null_2012 = Deaths2_group_2012[Deaths2_group_2012["Deaths2"]!=0].copy()
Deaths2_non_null_2012.sort_values(["Country","Deaths2"], ascending=False,inplace=True)
Deaths2_non_null_2012=Deaths2_non_null_2012.reset_index()
Result_2012 = Deaths2_non_null_2012.groupby(["Country"]).head(4)
Result_2012 = Result_2012.reset_index()

Result_2012["Infant Death %"] = 0

for i, row in Result_2012.iterrows():
   if row['Cause'] == "AAA":
       overall_deaths = row["Deaths2"]
       Result_2012.iloc[i, 4] = 1
   else:
       Result_2012.iloc[i, 4] = row["Deaths2"]/overall_deaths

Result_2012.head(20)